In [53]:
!pip install pandas pyvis networkx

# Считываем данные

In [21]:
import pandas as pd
from pyvis.network import Network

sheet_name = 'Export lexemes'
lexeme = pd.read_excel('lexeme_table.xlsx', sheet_name=sheet_name)
etymology = pd.read_excel('etymology_and_lexeme_table.xlsx', sheet_name=None)

# Настраиваем связи

In [4]:
merged_df = pd.merge(
    etymology['Lexemes'][['id', 'Лексема']],
    etymology['Lexemes and parent lexemes'][['Id лексемы', 'Id производящей лексемы']],
    left_on='id',
    right_on='Id лексемы',
    how='left'
)[['id', 'Id производящей лексемы', 'Лексема']]

merged_df = pd.merge(
    merged_df,
    etymology['Lexemes'][['id', 'Лексема']].rename(columns={'Лексема': 'Производящая лексема', 'id': 'id_m'}),
    left_on='Id производящей лексемы',
    right_on='id_m',
    how='left'
)[['id', 'Id производящей лексемы', 'Лексема', 'Производящая лексема']]

merged_df = pd.merge(
    merged_df,
    etymology['Lexemes and etymologies'],
    left_on='id',
    right_on='Id лексемы',
    how='left'
)[['id', 'Id производящей лексемы', 'Лексема', 'Производящая лексема', 'Id этимологии']]

merged_df = pd.merge(
    merged_df,
    etymology['Etymologies'].rename(columns={'id': 'id_m'}),
    left_on='Id этимологии',
    right_on='id_m',
    how='left'
)[['id', 'Id производящей лексемы', 'Лексема', 'Производящая лексема', 'Id этимологии', 'Этимон']]


merged_df = merged_df[merged_df['id'] > 0]
merged_df.head()

,id,Id производящей лексемы,Лексема,Производящая лексема,Id этимологии,Этимон
0,5880,NaN,-,NaN,NaN,NaN
1,9322,NaN,<...>асивое семя,NaN,NaN,NaN
2,9314,NaN,<...>геилъ,NaN,NaN,NaN
3,9324,NaN,<...>унелное семя,NaN,NaN,NaN
4,9323,NaN,<ф(е/и)>никулное семя,NaN,NaN,NaN


In [5]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12546 entries, 0 to 12545
Data columns (total 6 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   id                       12546 non-null  int64  
 1   Id производящей лексемы  4101 non-null   float64
 2   Лексема                  12546 non-null  object 
 3   Производящая лексема     4101 non-null   object 
 4   Id этимологии            1002 non-null   float64
 5   Этимон                   1002 non-null   object 
dtypes: float64(2), int64(1), object(3)
memory usage: 588.2+ KB


In [6]:
etymology_df = pd.merge(
    etymology['Etymologies'][['id', 'Этимон']],
    etymology['Etymologies and parent ety...'][['Id этимологии', 'Id производящей этимологии']],
    left_on='id',
    right_on='Id этимологии',
    how='left'
)[['id', 'Id производящей этимологии', 'Этимон']]

etymology_df = pd.merge(
    etymology_df,
    etymology['Etymologies'][['id', 'Этимон']].rename(columns={'Этимон': 'Производящий Этимон', 'id': 'id_m'}),
    left_on='Id производящей этимологии',
    right_on='id_m',
    how='left'
)[['id', 'Id производящей этимологии', 'Этимон', 'Производящий Этимон']]
etymology_df = etymology_df[etymology_df['id'] > 0]
etymology_df.head()

,id,Id производящей этимологии,Этимон,Производящий Этимон
0,500,NaN,(mel)arancia,NaN
1,37,NaN,*abl-on-,NaN
2,36,NaN,*ablonь,NaN
3,42,41.0,*ablъko,*āblu-
4,748,NaN,*agnę,NaN


# Строим граф

In [7]:
import networkx as nx
import json

nodes = []
edges = []
row_id = 1
G = nx.Graph()

for _, row in merged_df.iterrows():
    if f'lexeme_{row["id"]}' not in [s['id'] for s in nodes]:
        G.add_node(f"lexeme_{row['id']}", label=f"Lexeme {row['Лексема']}")
        nodes.append({"id": f"lexeme_{row['id']}", "label": f"Lexeme {row['Лексема']}", "size": 1, "color": "#0074D9"})

for _, row in etymology_df.iterrows():
    if f'etymology_{row["id"]}' not in [s['id'] for s in nodes]:
        G.add_node(f"etymology_{row['id']}", label=f"Etymology {row['Этимон']}")
        nodes.append({'id': f"etymology_{row['id']}", "label": f"Etymology {row['Этимон']}"})

for _, row in merged_df.iterrows():
    if pd.notna(row['Id производящей лексемы']) and (f"lexeme_{row['id']}", f"lexeme_{int(row['Id производящей лексемы'])}") not in [(s['source'], s['target']) for s in edges]:
        G.add_edge(f"lexeme_{row['id']}", f"lexeme_{int(row['Id производящей лексемы'])}")    
        edges.append({'id': f'row_{row_id}', "source": f"lexeme_{row['id']}", "target": f"lexeme_{int(row['Id производящей лексемы'])}", "label": f"Производящая лексема: {row['Производящая лексема']}", "size": 1, "color": "#0074D9"})
        row_id += 1
    
    if pd.notna(row['Id этимологии']) and (f"lexeme_{row['id']}", f"etymology_{int(row['Id этимологии'])}") not in [(s['source'], s['target']) for s in edges]:
        G.add_edge(f"lexeme_{row['id']}", f"etymology_{int(row['Id этимологии'])}")    
        edges.append({'id': f'row_{row_id}', "source": f"lexeme_{row['id']}", "target": f"etymology_{int(row['Id этимологии'])}", "label": f"Этимология {row['Этимон']}", "size": 1, "color": "#0074D9"})
        row_id += 1

for _, row in etymology_df.iterrows():
    if pd.notna(row['Id производящей этимологии']) and (f"etymology_{row['id']}", f"etymology_{int(row['Id производящей этимологии'])}") not in [(s['source'], s['target']) for s in edges]:
        G.add_edge(f"etymology_{row['id']}", f"etymology_{int(row['Id производящей этимологии'])}")    
        edges.append({'id': f'row_{row_id}', "source": f"etymology_{row['id']}", "target": f"etymology_{int(row['Id производящей этимологии'])}", "label": f"Производящий Этимон: {row['Производящий Этимон']}", "size": 1, "color": "#0074D9"})
        row_id += 1

graph_data = {"nodes": nodes, "edges": edges}
with open('graph_data.json', 'w', encoding='utf-8') as f:
    json.dump(graph_data, f, ensure_ascii=False, indent=4)

In [36]:
def get_neighbors(graph, node, deep):
    neighbors = set()
    neighbors.add(node)
    neighbors.update(graph.neighbors(node))
    while deep >= 2:
        nodes = list(neighbors)
        for node in nodes:
            neighbors.update(graph.neighbors(node))
        deep -= 1
    return neighbors

def get_top_nodes_by_degree(graph, n):
    degree_dict = dict(graph.degree())
    top_nodes = sorted(degree_dict, key=degree_dict.get, reverse=True)[:n]
    return top_nodes

def build_subgraph_from_top_nodes(graph, nodes):
    nodes_to_explore = set(nodes)
    visited_nodes = set()

    while nodes_to_explore:
        node = nodes_to_explore.pop()
        if node not in visited_nodes:
            visited_nodes.add(node)
            neighbors = set(graph.neighbors(node))
            nodes_to_explore.update(neighbors - visited_nodes)

    subgraph = graph.subgraph(visited_nodes)
    return subgraph

# Задача - Уметь выделять подграф нод со связами

In [37]:
top_nodes = get_top_nodes_by_degree(G, 20)
print(*[G.nodes[node]['label'] for node in top_nodes], sep='\n')

Lexeme трава
Lexeme дерево
Lexeme роза
Lexeme дубъ
Lexeme дикий
Lexeme орѣхъ
Lexeme бѣлый
Lexeme дрѣво
Lexeme капуста
Lexeme яблоко
Lexeme горохъ
Lexeme бальсамъ
Lexeme корень
Lexeme лукъ
Lexeme большой
Lexeme вишня
Lexeme бобъ
Lexeme красный
Lexeme ленъ
Lexeme щавель


In [51]:
lexeme_word = input('Введите лексему: ')
deep = input('Глубина просмотра: ')
net = Network(cdn_resources='remote', height='1000px', width='1500px')
node_id = [node for node in G.nodes if G.nodes[node]['label'] == f'Lexeme {lexeme_word}']
if not node_id:
    print('Такой лексемы не нашлось')
else:
    try:
        deep = int(deep)
    except:
        print('Нужно было ввести число... по умолчанию поставил 2')
        deep = 2
    subgraph = G.subgraph(get_neighbors(G, node_id[0], deep))
    net.from_nx(subgraph)
    net.show("graph.html")

Введите лексему:  корень
Глубина просмотра:  2
